# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [68]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

from pyspark.sql.functions import col, isnan, when, count, round, mean

  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 2c519c3a-d5df-4fce-bb95-9896f8af61c5.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 2c519c3a-d5df-4fce-bb95-9896f8af61c5.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 3.0


You are already connected to a glueetl session 2c519c3a-d5df-4fce-bb95-9896f8af61c5.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 2c519c3a-d5df-4fce-bb95-9896f8af61c5.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



In [54]:
s3_path = 's3://tekraj-test2/cigna-test/Churn/charges.csv'

In [55]:
df = spark.read.csv(s3_path, header= True)

In [46]:
df.show(10)

+----------+------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|tenure|      contract|paperlessBilling|       paymentMethod|monthlyCharges|totalCharges|churn|
+----------+------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|     1|Month-to-month|             Yes|    Electronic check|         29.85|       29.85|   No|
|5575-GNVDE|    34|      One year|              No|        Mailed check|         56.95|      1889.5|   No|
|3668-QPYBK|     2|Month-to-month|             Yes|        Mailed check|         53.85|      108.15|  Yes|
|7795-CFOCW|    45|      One year|              No|Bank transfer (au...|          42.3|     1840.75|   No|
|9237-HQITU|     2|Month-to-month|             Yes|    Electronic check|          70.7|      151.65|  Yes|
|9305-CDSKC|     8|Month-to-month|             Yes|    Electronic check|         99.65|       820.5|  Yes|
|1452-KIOVK|    22|Month-to-month|   

In [49]:
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- paperlessBilling: string (nullable = true)
 |-- paymentMethod: string (nullable = true)
 |-- monthlyCharges: string (nullable = true)
 |-- totalCharges: string (nullable = true)
 |-- churn: string (nullable = true)


In [67]:
# Checking null counts in all the columns
df.select([count(when(col(column).isNull() | (col(column)=='NA') | isnan(column), column)).alias(column) for column in df.columns]).show()

+----------+------+--------+----------------+-------------+--------------+------------+-----+
|customerID|tenure|contract|paperlessBilling|paymentMethod|monthlyCharges|totalCharges|churn|
+----------+------+--------+----------------+-------------+--------------+------------+-----+
|         0|     0|       0|               0|            0|           455|         455|    0|
+----------+------+--------+----------------+-------------+--------------+------------+-----+


In [95]:
df = df.withColumn("monthlyCharges", col("monthlyCharges").cast("float"))
df = df.withColumn("totalCharges", col("totalCharges").cast("float"))
df = df.withColumn("tenure", col("tenure").cast("float"))

In [75]:
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- paperlessBilling: string (nullable = true)
 |-- paymentMethod: string (nullable = true)
 |-- monthlyCharges: double (nullable = true)
 |-- totalCharges: double (nullable = true)
 |-- churn: string (nullable = true)


In [81]:
trim_perc = 0.1

In [96]:
lower_trim, upper_trim = df.approxQuantile("monthlyCharges", [trim_perc/2, 1 - trim_perc/2], 0)

In [97]:
trimmed_df = df.filter((col("monthlyCharges") >= lower_trim) & (col("monthlyCharges") <= upper_trim))

In [98]:
trimmed_avg = trimmed_df.select(mean(col("monthlyCharges"))).first()[0]

In [99]:
# 3. Filling missing values in monthlyCharges col
df = df.withColumn("monthlyCharges", when(col("monthlyCharges").isNull(), round(trimmed_avg)).otherwise(col("monthlyCharges")))

TypeError: Invalid argument, not a string or column: 64.50086309977124 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.


In [100]:
# 4. Filling Missing values in totalCharges column
df = df.withColumn("totalCharges", when(col("monthlyCharges").isNull(), round(col("monthlyCharges")/col("tenure"))).otherwise(col("totalCharges")))

In [101]:
df.show(30)

+----------+------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|tenure|      contract|paperlessBilling|       paymentMethod|monthlyCharges|totalCharges|churn|
+----------+------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|   1.0|Month-to-month|             Yes|    Electronic check|         29.85|        30.0|   No|
|5575-GNVDE|  34.0|      One year|              No|        Mailed check|         56.95|         2.0|   No|
|3668-QPYBK|   2.0|Month-to-month|             Yes|        Mailed check|         53.85|        27.0|  Yes|
|7795-CFOCW|  45.0|      One year|              No|Bank transfer (au...|          42.3|         1.0|   No|
|9237-HQITU|   2.0|Month-to-month|             Yes|    Electronic check|          70.7|        35.0|  Yes|
|9305-CDSKC|   8.0|Month-to-month|             Yes|    Electronic check|         99.65|        12.0|  Yes|
|1452-KIOVK|  22.0|Month-to-month|   

In [ ]:
#avg_value = df.select(mean(col("monthlyCharges"))).filter()[0]

In [ ]:
# 5. Calculating the Churn Rate

In [109]:
churn_df = df.filter(col("churn")=='Yes')

In [110]:
churn_counts = churn_df.count()
total_customers = df.count()

In [114]:
churn_rate = (churn_counts/total_customers)*100
print(churn_rate)

26.578498293515356


In [24]:

# 2nd Data - Personal

In [16]:
s3_path2 = 's3://tekraj-test2/cigna-test/Churn/personal.csv'

In [17]:
df2 = spark.read.csv(s3_path2, header = True, inferSchema = True)

In [37]:
null_cols = df2.select([count(when(col(column).isNull() | isnan(column), column)).alias(column) for column in df2.columns]).show()

+----------+------+-------+----------+---+
|customerID|gender|partner|dependents|age|
+----------+------+-------+----------+---+
|         0|     0|      0|         0|  0|
+----------+------+-------+----------+---+


In [115]:
df2.show(10)

+----------+------+-------+----------+---+
|customerID|gender|partner|dependents|age|
+----------+------+-------+----------+---+
|5575-GNVDE|  Male|     No|        No| 41|
|3668-QPYBK|  Male|     No|        No| 58|
|7795-CFOCW|  Male|     No|        No| 61|
|9237-HQITU|Female|     No|        No| 66|
|9305-CDSKC|Female|     No|        No| 87|
|1452-KIOVK|  Male|     No|       Yes| 39|
|6713-OKOMC|Female|     No|        No| 39|
|7892-POOKP|Female|    Yes|        No| 38|
|6388-TABGU|  Male|     No|       Yes| 50|
|9763-GRSKD|  Male|    Yes|       Yes| 50|
+----------+------+-------+----------+---+
only showing top 10 rows


In [116]:
df2.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- partner: string (nullable = true)
 |-- dependents: string (nullable = true)
 |-- age: integer (nullable = true)


In [22]:

# Question 6: Joined data
# Join the updated charges data with personal data and the plan data by the customerID.


In [118]:
joined_data = df.join(df2, "customerID", "inner")
joined_data.show()

+----------+------+--------------+----------------+--------------------+--------------+------------+-----+------+-------+----------+---+
|customerID|tenure|      contract|paperlessBilling|       paymentMethod|monthlyCharges|totalCharges|churn|gender|partner|dependents|age|
+----------+------+--------------+----------------+--------------------+--------------+------------+-----+------+-------+----------+---+
|5575-GNVDE|  34.0|      One year|              No|        Mailed check|         56.95|         2.0|   No|  Male|     No|        No| 41|
|3668-QPYBK|   2.0|Month-to-month|             Yes|        Mailed check|         53.85|        27.0|  Yes|  Male|     No|        No| 58|
|7795-CFOCW|  45.0|      One year|              No|Bank transfer (au...|          42.3|         1.0|   No|  Male|     No|        No| 61|
|9237-HQITU|   2.0|Month-to-month|             Yes|    Electronic check|          70.7|        35.0|  Yes|Female|     No|        No| 66|
|9305-CDSKC|   8.0|Month-to-month|       

In [23]:
#Question 7: Calculate the percentage of customers more than 65 years old

In [121]:
older_than_65 = joined_data.filter(col("age") > 65).count()

In [119]:
total_customers_new = joined_data.count()

In [125]:
percent_above_65 = (older_than_65 / total_customers_new) * 100
print(percent_above_65)

34.874905231235786


In [ ]:
# 3rd Data - Plan

In [20]:
s3_path3 = 's3://tekraj-test2/cigna-test/Churn/plan.csv'

In [24]:
df3 = spark.read.csv(s3_path3, header = True, inferSchema = True)

In [38]:
null_cols = df3.select([count(when(col(column).isNull() | isnan(column), column)).alias(column) for column in df3.columns]).show()

+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+
|customerID|phoneService|multipleLines|internetService|onlineSecurity|onlineBackup|deviceProtection|techSupport|streamingTV|streamingMovies|
+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+
|         0|           0|            0|              0|             0|           0|               0|          0|          0|              0|
+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+


In [25]:
df3.show(10)

+----------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|customerID|phoneService|   multipleLines|internetService|     onlineSecurity|       onlineBackup|   deviceProtection|        techSupport|        streamingTV|    streamingMovies|
+----------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|5575-GNVDE|         Yes|              No|            DSL|                Yes|                 No|                Yes|                 No|                 No|                 No|
|7795-CFOCW|          No|No phone service|            DSL|                Yes|                 No|                Yes|                Yes|                 No|                 No|
|9237-HQITU|         Yes|              No|    Fiber optic|                 No|                 No|       

In [26]:
df3.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- phoneService: string (nullable = true)
 |-- multipleLines: string (nullable = true)
 |-- internetService: string (nullable = true)
 |-- onlineSecurity: string (nullable = true)
 |-- onlineBackup: string (nullable = true)
 |-- deviceProtection: string (nullable = true)
 |-- techSupport: string (nullable = true)
 |-- streamingTV: string (nullable = true)
 |-- streamingMovies: string (nullable = true)
